# QMSS 301 Web Scarping - Reddit

In [1]:
# This part is to import the packages for later analysis. 
import requests
import pandas as pd

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# **API Security Tokens**

In [2]:
# This part is to pass login method, user name and passowrd 
auth = requests.auth.HTTPBasicAuth('FUieF7xEkLsimc79KGxSjA', 'uuiREkQQT0HknLLNCMHp8j_l8j4Byw')

data = {'grant_type': 'password',
        'username': 'Lucky_Race_4947',
        'password': '@Lyx990621.'}

In [3]:
# This part is to setup header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# This is to send  request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# This is to convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# This is to add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}


# **Subreddit Page Details**

In [4]:
# This is to make a request for the trending posts in /r/Health

subreddit = "health"  # subreddit page title - Eg - 
type_post = "best"  # controversial, best, hot, new, random, rising, top
limit = 100   # default: 25; Max: 100
num_pages = 70 # number of pages

# **API Call - Simple Example**

In [ ]:
# This is to extract at least 5000+ posts
response = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{type_post}.json?limit={limit}".format(subreddit, type_post, limit),  
                   headers=headers)

print(response.json()['data']['children'])

**Understanding the Reddit Data**

https://jsonformatter.curiousconcept.com/

# **API Call - Pagination**

**The response will contain an "after" value which you can pass in the next request.**

In [6]:
# This is pagination
url = f"https://oauth.reddit.com/r/{subreddit}/{type_post}.json?limit={limit}".format(subreddit, type_post, limit)
posts = []
after = '' # Default/Initial value
before = '' # Default/Initial value

for page in range(0, num_pages): 
    page_url = url
    if before:
        page_url += '&before=' + before  # before key for the before page (P - 1) -> P is the current page
    if after:
        page_url += '&after=' + after  # after key for the next page (P + 1)
    
    response = requests.get(page_url, headers=headers).json()
    posts += response['data']['children']
    after = response['data']['after']  # Extracting the after field from JSON Data
    before = response['data']['before']  # Extracting the before field from JSON Data



# **Reddit JSON Data Keys**

In [8]:
# This is to printing all the variable keys in the data and have a better understnading of the data structure 
for key in response['data']['children'][0]['data'].keys():
    print(key) 



approved_at_utc
subreddit
selftext
author_fullname
saved
mod_reason_title
gilded
clicked
title
link_flair_richtext
subreddit_name_prefixed
hidden
pwls
link_flair_css_class
downs
thumbnail_height
top_awarded_type
hide_score
name
quarantine
link_flair_text_color
upvote_ratio
author_flair_background_color
subreddit_type
ups
total_awards_received
media_embed
thumbnail_width
author_flair_template_id
is_original_content
user_reports
secure_media
is_reddit_media_domain
is_meta
category
secure_media_embed
link_flair_text
can_mod_post
score
approved_by
is_created_from_ads_ui
author_premium
thumbnail
edited
author_flair_css_class
author_flair_richtext
gildings
post_hint
content_categories
is_self
mod_note
created
link_flair_type
wls
removed_by_category
banned_by
author_flair_type
domain
allow_live_comments
selftext_html
likes
suggested_sort
banned_at_utc
url_overridden_by_dest
view_count
archived
no_follow
is_crosspostable
pinned
over_18
preview
all_awardings
awarders
media_only
can_gild
spoiler

# **JSON Data Array to Pandas Dataframe**

In [11]:
# This is to initialize the data frmae and append the data points into the datafromae 
df = pd.DataFrame()  

for post in posts:
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'], 
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score'],
        'total_awards_received': post['data']['total_awards_received'], 
        'num_comments': post['data']['num_comments'],
        'name': post ['data']['name']
    }, ignore_index=True)

df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,total_awards_received,num_comments,name
0,Health,9 million children to be vaccinated against po...,,0.97,227,0,227,1,9,t3_tm4duo
1,Health,Male birth control pill 99 per cent effective ...,,0.77,9,0,9,0,1,t3_tmvnpp
2,Health,FDA: Unsanitary Conditions Found at Baby Food ...,,1.00,8,0,8,0,1,t3_tmqs4n
3,Health,California governor signs law that makes abort...,,0.95,791,0,791,0,20,t3_tkwola
4,Health,Federal judge sides with 12 disabled kids seek...,,0.88,6,0,6,0,0,t3_tmvkjk


# **Saving the data** 

In [12]:
# This is to save the data into a csv file 
df.to_csv('./data1.csv', index=True, header=True)